# Google

In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
# install with pip: transformers, sentencepiece, google, protobuf
# conda install -c conda-forge ipywidgets

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=True`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [7]:
input_text = "You are to fine-tune a text prompt for StyleCLIP, which is a model built on StyleGAN and CLIP to manipulate images based on text prompts. The StyleGAN model is pretrained on face dataset and the input text prompt is 'a sad car'. Suggest a text prompt that would work better on this situation."
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

outputs = model.generate(input_ids, max_length=100)
print(tokenizer.decode(outputs[0]))

<pad> a sad car</s>


In [8]:
tokenizer.decode(outputs[0])

'<pad> a sad car</s>'

# LLaMa 2, 7B

In [9]:
import subprocess, time

In [11]:
def run(ckpt, task):
    """
    Runs a terminal command and prints the output followed by error message.
    Args:
        ckpt (str): Path to LLaMa model checkpoint
        task (str): Path to Python task
    Returns:
        float: Command execution time
    """
    start = time.time()
    cmd = f"torchrun --nproc_per_node 1 {task} --ckpt_dir {ckpt}/ --tokenizer_path tokenizer.model --max_seq_len 512 --max_batch_size 6"
    result = subprocess.run(cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    end = time.time()
    exec = end-start
    print(f"Output:\n{result.stdout}")
    print(f"Error:\n{result.stderr}")
    print(f"Execution time: {exec} seconds")
    return exec 

In [12]:
run('../llama/llama-2-7b', '../llama/example_text_completion.py')

CalledProcessError: Command 'torchrun --nproc_per_node 1 ../llama/example_text_completion.py --ckpt_dir ../llama/llama-2-7b/ --tokenizer_path tokenizer.model --max_seq_len 512 --max_batch_size 6' returned non-zero exit status 1.

In [13]:
!torchrun --nproc_per_node 1 ../llama/example_text_completion.py --ckpt_dir ../llama/llama-2-7b/ --tokenizer_path tokenizer.model --max_seq_len 512 --max_batch_size 6

Traceback (most recent call last):
  File "/root/bramyeon/StyleCLIP/../llama/example_text_completion.py", line 69, in <module>
    fire.Fire(main)
  File "/root/anaconda3/lib/python3.11/site-packages/fire/core.py", line 141, in Fire
    component_trace = _Fire(component, args, parsed_flag_args, context, name)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/root/anaconda3/lib/python3.11/site-packages/fire/core.py", line 475, in _Fire
    component, remaining_args = _CallAndUpdateTrace(
                                ^^^^^^^^^^^^^^^^^^^^
  File "/root/anaconda3/lib/python3.11/site-packages/fire/core.py", line 691, in _CallAndUpdateTrace
    component = fn(*varargs, **kwargs)
                ^^^^^^^^^^^^^^^^^^^^^^
  File "/root/bramyeon/StyleCLIP/../llama/example_text_completion.py", line 32, in main
    generator = Llama.build(
                ^^^^^^^^^^^^
  File "/root/bramyeon/llama/llama/generation.py", line 85, in build
    torch.distributed.in

In [14]:
!python -m torch.utils.collect_env

PyTorch version: 2.1.0
Is debug build: False
CUDA used to build PyTorch: Could not collect
ROCM used to build PyTorch: N/A

OS: Ubuntu 22.04.2 LTS (x86_64)
GCC version: (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
Clang version: Could not collect
CMake version: Could not collect
Libc version: glibc-2.35

Python version: 3.11.5 (main, Sep 11 2023, 13:54:46) [GCC 11.2.0] (64-bit runtime)
Python platform: Linux-5.15.0-67-generic-x86_64-with-glibc2.35
Is CUDA available: False
CUDA runtime version: 11.5.119
CUDA_MODULE_LOADING set to: N/A
GPU models and configuration: Could not collect
Nvidia driver version: Could not collect
cuDNN version: Could not collect
HIP runtime version: N/A
MIOpen runtime version: N/A
Is XNNPACK available: True

CPU:
Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Address sizes:                   46 bits physical, 48 bits virtual
Byte Order:                      Little Endian
CPU(s):                          40
On-line CPU(s) list: 

## Dolly
https://github.com/databrickslabs/dolly#getting-started-with-response-generation

In [1]:
from transformers import pipeline
import torch

instruct_pipeline = pipeline(model="databricks/dolly-v2-12b", torch_dtype=torch.bfloat16, trust_remote_code=True, device_map="auto")

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


KeyboardInterrupt: 

In [ ]:
instruct_pipeline("Explain to me the difference between nuclear fission and fusion.")